In [ ]:
# import dependencies
import dateutil.parser as parser
import pandas as pd
import os

In [ ]:
# list of raw data files
raw_data = os.listdir("raw_data")

# list of dataframes
df_list = []

# specify row parameters
target_rows = 500000
rows_per_file = int(target_rows / len(raw_data))

# specify target column names
target_columns = ["trip_duration",
                  "start_time",
                  "stop_time",
                  "start_station_id",
                  "start_station_name",
                  "start_station_lat",
                  "start_station_lng",
                  "end_station_id",
                  "end_station_name",
                  "end_station_lat",
                  "end_station_lng",
                  "bike_id",
                  "user_type",
                  "birth_year",
                  "gender"]

# iterate through the data files
i = 0
for file in raw_data:
    
    # read the csv into a dataframe
    raw_df = pd.read_csv(os.path.join("raw_data", file))
    
    # store the ride count
    ride_count = raw_df.shape[0]
    
    # drop null rows
    raw_df = raw_df.dropna()
    
    # rename the columns
    raw_df.columns = target_columns
    
    # remove messy newline characters
    raw_df = raw_df.replace("\\N", "")
    
    # only allow numeric rows and enfore birth year data type
    raw_df = raw_df.loc[(raw_df["birth_year"] != "")]
    raw_df["birth_year"] = raw_df["birth_year"].astype(int)
    
    # remove birth year typos
    raw_df = raw_df.loc[(raw_df["birth_year"] > 1920)]
    
    # extract the required rows
    if raw_df.shape[0] > rows_per_file:
        filtered_df = raw_df.sample(n = rows_per_file)
    
    # enforce datetime data types
    filtered_df["start_time"] = filtered_df["start_time"].apply(lambda x: parser.parse(x).strftime("%Y/%m/%d %H:%M:%S"))
    filtered_df["stop_time"] = filtered_df["stop_time"].apply(lambda x: parser.parse(x).strftime("%Y/%m/%d %H:%M:%S"))
    
    # save the current year for this dataset
    current_year = parser.parse(filtered_df["start_time"].iloc[0]).year
    
    # create a new column for user age
    filtered_df["user_age"] = filtered_df.apply(lambda x: current_year - x["birth_year"], axis = 1)
    
    # create a new column for total ride counts
    filtered_df["ride_count"] = ride_count
    
    # add dataframe to the list
    print(f"{i}\t{file}")
    i += 1
    df_list.append(filtered_df)

# construct the final df
main_df = pd.concat(df_list)
main_df

In [ ]:
# save the dataframes to file
main_df.to_csv(os.path.join("data", "bike_data.csv"), index = False)